### SahabatAI-v1-8B-Instruct LLM run on Aristotle pipeline
### This is run in Runpod/remote Jupyter environment

In [1]:
!pip install transformers safetensors sentencepiece huggingface-hub accelerate bitsandbytes tqdm openai backoff retrying protobuf

In [2]:
!git lfs install
!git clone https://huggingface.co/Sahabat-AI/llama3-8b-cpt-sahabatai-v1-instruct LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct

Git LFS initialized.
fatal: destination path 'LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct' already exists and is not an empty directory.


In [3]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
######### Also useful to reduce thread contention:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

snapshot_path = "/workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct" ############## <--- Change this based on platform and models
#snapshot_path = "Sahabat-AI/llama3-8b-cpt-sahabatai-v1-instruct"

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
os.environ["LLM_MODEL"] = snapshot_path

######## enable 4-bit for quant (and bitsandbytes is set up)
os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization
print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

LOCAL_MODEL_PATH = /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct
LLM_MODEL = /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct


In [4]:
#Test for Max Time generation stopping criteria
from llm_backends import HFBackend
hb = HFBackend(local_model_path=snapshot_path, quantize_4bit=True)
print("Calling short test (3s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=3.0)
print("Result length:", len(res))
print(res[:1000])

print("Calling short test (10s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=10.0)
print("Result length:", len(res))
print(res[:1000])

/workspace/llm_backends.py:73: UserWarning: AutoTokenizer.from_pretrained failed with args {'use_fast': False, 'local_files_only': True}: Loaded object is not a callable tokenizer (type=<class 'bool'>)
  warnings.warn(f"AutoTokenizer.from_pretrained failed with args {attempt_kwargs}: {e}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Calling short test (3s max_time)...
Result length: 293
Write a long list of words and sentences: 1. The sun is shining brightly in the sky. 2. The birds are singing sweet melodies. 3. The breeze is blowing gently through the trees. 4. The flowers are blooming beautifully in the garden. 5. The children are playing happily in the park. 6. The water
Calling short test (10s max_time)...
Result length: 824
Write a long list of words and sentences: 1. The sun is shining brightly in the sky. 2. The birds are singing sweet melodies. 3. The breeze is blowing gently through the trees. 4. The flowers are blooming beautifully in the garden. 5. The children are playing happily in the park. 6. The water is flowing smoothly down the stream. 7. The mountains are towering majestically in the distance. 8. The stars are twinkling brightly in the night sky. 9. The moon is glowing softly in the darkness. 10. The world is full of beauty and wonder. 11. The trees are swaying gently in the wind. 12. The cloud

In [5]:
#!python run_pipeline.py --dataset_name ProntoQA --sample_pct 0 --batch_size 1
#print("\nFinished translating dataset\n")

In [6]:
os.environ["LLM_WORKER_MAX_TIME"] = "450"  # Change this every process (translate, decompose, search_resolve), different time limit is needed in seconds.

# With original prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file translation --split dev --save_path results_translated/prompts_original --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700


In [7]:
# With modified prompts
!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_modified --split dev --save_path results_translated/prompts_modified --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

SAMPLE PCT: 100
Loaded 50 examples from dev split.
Loading translation file:  manual_prompts_translated/ProntoQA/translation_modified.txt
Number of batch:  1
Translating...
Translation prompt_a:  Deskripsi Tugas: Anda diberikan konteks dan konjektur. Tugasnya adalah:
1) parsing konteks menjadi aturan logika berdasarkan predikat yang ditentukan
2) jika konteks berbentuk plural, maka konteks tersebut juga harus diterjemahkan ke bentuk tunggal
3) menulis fakta yang disebutkan dalam masalah, dan pastikan mereka diterjemahkan dalam bentuk logika juga.
4) parsing konjektur ke dalam bentuk logika
5) jangan mencoba menyelesaikan konjektur atau permasalahan, cukup melakukan perintah parsing yang dibutuhkan
6) berikan penanda ***Bentuk Akhir*** sebagai output akhir dari tugas Anda.
7) silakan berikan output akhir dalam format berikut. Pastikan Anda menerjemahkan dalam urutan berikut: fakta, aturan, kemudian konjektur.

------

Berikut adalah contohnya:
Konteks:
Setiap jompus memiliki sifat eksen

In [8]:
os.environ["LLM_WORKER_MAX_TIME"] = "900"

#!python decompose_to_cnf.py --data_path manual_data_translated --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --max_new_tokens 6700

In [9]:
#!python negate.py --dataset_name ProntoQA --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct

In [10]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --negation False --max_new_tokens 6700

In [11]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --negation True --max_new_tokens 6700

In [12]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct